In [5]:
import json
from datetime import datetime

filename = 'corona-out-2'
#filename = './../corona-out-3'

In [6]:
def to_timestamp(val):
    dt = val.split(' ')
    st = f"{dt[5]}-{dt[1]}-{dt[2]} {dt[3]}"
    dt = datetime.strptime(st, "%Y-%b-%d %H:%M:%S")
    return int(dt.timestamp())

def to_dtstring(val):
    return datetime.fromtimestamp(val).strftime('%b %d, %Y - %X')

def get_hashtags(data):
    hashtags = []
    for item in data:
        hashtags.append(item['text'])
    return hashtags

In [7]:
# def add_user(user):
#     if (user['id'] not in users):
#         users[user['id']] = user

# def add_tweet(tweet, tweet_type):
#     if tweet_type=='tweet':
#         if tweet['id'] not in tweets:
#             tweets[tweet['id']] = tweet
#     elif tweet_type=='retweet':
#         if tweet['id'] not in retweets:
#             retweets[tweet['id']] = tweet

# def add_retweet(retweet):
#     if (retweet['id'] not in retweets):
#         retweets[retweet['id']] = retweet

def add_user(user):
    if (user['id'] not in users):
        new_user = {}
        
        new_user['id'] = user['id']
        new_user['name'] = user['name']
        new_user['screen_name'] = user['screen_name']
        new_user['location'] = user['location']
        new_user['url'] = user['url']
        new_user['description'] = user['description']
        new_user['verified'] = user['verified']
        new_user['followers_count'] = user['followers_count']
        new_user['friends_count'] = user['friends_count']
        new_user['listed_count'] = user['listed_count']
        new_user['favourites_count'] = user['favourites_count']
        new_user['statuses_count'] = user['statuses_count']
        new_user['created_at'] = to_timestamp(user['created_at'])
        
        users[user['id']] = new_user
        


def process_tweet(tweet):
    new_tweet = {}
        
    new_tweet['created_at'] = to_timestamp(tweet['created_at'])
    new_tweet['id'] = tweet['id']
    if tweet['truncated']:
        new_tweet['text'] = tweet['extended_tweet']['full_text']
        new_tweet['hashtags'] = get_hashtags(tweet['extended_tweet']['entities']['hashtags'])
    else:
        new_tweet['text'] = tweet['text']
        new_tweet['hashtags'] = get_hashtags(tweet['entities']['hashtags'])

    user = {}
    user['id'] = tweet['user']['id']
    user['name'] = tweet['user']['name']
    user['screen_name'] = tweet['user']['screen_name']
    user['verified'] = tweet['user']['verified']
    new_tweet['user'] = user

    new_tweet['source'] = tweet['source']
    new_tweet['quote_count'] = tweet['quote_count']
    new_tweet['reply_count'] = tweet['reply_count']
    new_tweet['retweet_count'] = tweet['retweet_count']
    new_tweet['favorite_count'] = tweet['favorite_count']
    
    if 'retweeted_status' in tweet:
        new_tweet['retweeted_status'] = process_tweet(tweet['retweeted_status'])
        
    new_tweet['is_quote_status'] = tweet['is_quote_status']
    if tweet['is_quote_status'] and 'quoted_status' in tweet:
        new_tweet['quoted_status'] = process_tweet(tweet['quoted_status'])
    
    return new_tweet
        
def add_tweet(tweet, tweet_type):
    if (tweet['id'] not in tweets):
        
        new_tweet = process_tweet(tweet)
        
        if tweet_type=='tweet':
            tweets[tweet['id']] = new_tweet #CHANGED
        elif tweet_type=='retweet':
            retweets[tweet['id']] = new_tweet #CHANGED
        elif tweet_type=='quote':
            quotes[tweet['id']] = new_tweet #CHANGED

In [8]:
def process_data(data):
    if data['id'] in tweets:
        return
    
    add_user(data['user'])

    if data['text'].startswith('RT'):
        add_tweet(data, 'retweet')
        if 'retweeted_status' in data:
            process_data(data['retweeted_status'])

    else:
        add_tweet(data, 'tweet')

In [9]:
users = {}
tweets = {}
retweets = {}
quotes = {}

for line in open(filename, 'r'):
    if len(line.strip()) == 0:
        continue
        
#     try:
    data = json.loads(line)
    process_data(data)

#     except:
#         continue

In [10]:
# (7349, 11157, 15505)
# (10987, 11157, 18461)

len(tweets), len(retweets), len(users), len(quotes)

(10987, 11157, 18461, 0)

In [11]:
retweets.keys()

dict_keys([1249403767180668930, 1249403768023678982, 1249403769193779202, 1249403769567227906, 1249403770435493888, 1249403770833981440, 1249403771077005315, 1249403771676815361, 1249403773178560515, 1249403773476356096, 1249403773522337792, 1249403773740552193, 1249403774151659521, 1249403775229534209, 1249403775493816321, 1249403775632076801, 1249403776441737216, 1249403777498542081, 1249403777804840962, 1249403778241044483, 1249403778895314956, 1249403779440463872, 1249403779692335106, 1249403779906035718, 1249403780661035008, 1249403780971585536, 1249403781240029189, 1249403781579771906, 1249403781575405568, 1249403783194357760, 1249403783458754560, 1249403783932772358, 1249403784624844800, 1249403785484664832, 1249403785820110854, 1249403785832738816, 1249403785849561091, 1249403785954430977, 1249403786562428929, 1249403786562621445, 1249403786759507968, 1249403787262881792, 1249403787250458628, 1249403788256874496, 1249403788433207297, 1249403789410533378, 1249403789343219713, 12

In [12]:
idxs = list(retweets.keys())

i = 0
for idx in idxs:
    if i==10: break
    print(retweets[idx]['text'])
    print(retweets[idx]['retweeted_status']['text'])
    print()
    i+=1

RT @nuffsaidny: wishing death on people is weirdo behavior.
wishing death on people is weirdo behavior.

RT @lale_karanfil: In Turkey, there are 300 thousand prisoners and 150 thousand prison employees in prisons. 
Prisons are the most risky pl…
In Turkey, there are 300 thousand prisoners and 150 thousand prison employees in prisons. 
Prisons are the most risky places in case of corona virus.
The lives of thousands of people are at risk.

#COVID19InTurkeysPrisons

@SusannaCeccardi

RT @umesh_agr: Thank You Sir !!
Really thankful for encouraging the 
@BSNL_OR 
 Staff risking their life in maintaining the Telecom Service…
Thank You Sir !!
Really thankful for encouraging the 
@BSNL_OR 
 Staff risking their life in maintaining the Telecom Services so that people can stay home &amp; work from home too.
#BSNL Odisha is doing great job.🙏🙏

RT @meysimek: Turkey is so stubborn to change their mind, they want innocent babies and their mothers, journalists, lawyers, doctors, human…
Turkey is so s

In [13]:
retweets[idxs[1]]

{'created_at': 1586730445,
 'id': 1249403768023678982,
 'text': 'RT @lale_karanfil: In Turkey, there are 300 thousand prisoners and 150 thousand prison employees in prisons. \nPrisons are the most risky pl…',
 'hashtags': [],
 'user': {'id': 1225145123920588805,
  'name': 'efe09',
  'screen_name': 'efe0927183508',
  'verified': False},
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'quote_count': 0,
 'reply_count': 0,
 'retweet_count': 0,
 'favorite_count': 0,
 'retweeted_status': {'created_at': 1586728961,
  'id': 1249397541596286979,
  'text': 'In Turkey, there are 300 thousand prisoners and 150 thousand prison employees in prisons. \nPrisons are the most risky places in case of corona virus.\nThe lives of thousands of people are at risk.\n\n#COVID19InTurkeysPrisons\n\n@SusannaCeccardi',
  'hashtags': ['COVID19InTurkeysPrisons'],
  'user': {'id': 1087735689091928064,
   'name': 'Karanfil Lale',
   'screen_name': 'lale_karanfil',
   'verified': 

In [14]:
retweets[idxs[0]].keys()

dict_keys(['created_at', 'id', 'text', 'hashtags', 'user', 'source', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'retweeted_status', 'is_quote_status', 'quoted_status'])

In [15]:
idxs = list(tweets.keys())

i = 0
for idx in idxs:
    if i==10: break
    print(tweets[idx])
    print()
    i+=1

{'created_at': 1586724481, 'id': 1249378751349231616, 'text': 'wishing death on people is weirdo behavior.', 'hashtags': [], 'user': {'id': 16144221, 'name': 'NUFF', 'screen_name': 'nuffsaidny', 'verified': False}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'quote_count': 1, 'reply_count': 1, 'retweet_count': 24, 'favorite_count': 50, 'is_quote_status': True, 'quoted_status': {'created_at': 1586709390, 'id': 1249315454797168641, 'text': 'I pray my haters die of corona virus 😷', 'hashtags': [], 'user': {'id': 46769281, 'name': 'Gucci Mane', 'screen_name': 'gucci1017', 'verified': True}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'quote_count': 3376, 'reply_count': 1956, 'retweet_count': 9246, 'favorite_count': 33006, 'is_quote_status': False}}

{'created_at': 1586728961, 'id': 1249397541596286979, 'text': 'In Turkey, there are 300 thousand prisoners and 150 thousand prison employees

In [16]:
tweets[idxs[1]].keys()

dict_keys(['created_at', 'id', 'text', 'hashtags', 'user', 'source', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'is_quote_status'])

In [17]:
users

{1242817830946508801: {'id': 1242817830946508801,
  'name': 'juwelz v',
  'screen_name': 'juwelz_v',
  'location': 'Lower East Side, Manhattan',
  'url': 'https://linktr.ee/juwelzv',
  'description': 'Event Lyfe LLC .. Brand Ambassador: #visionarysociety Music Manager: @ssizzzlee & @tmoneybeatsbang Production🎥📸 #BringNyCultureBack 🌊🌊🌊 !!',
  'verified': False,
  'followers_count': 43,
  'friends_count': 118,
  'listed_count': 0,
  'favourites_count': 722,
  'statuses_count': 906,
  'created_at': 1585160248},
 16144221: {'id': 16144221,
  'name': 'NUFF',
  'screen_name': 'nuffsaidny',
  'location': None,
  'url': None,
  'description': 'instagram: @nuffsaidny 🇹🇹',
  'verified': False,
  'followers_count': 17112,
  'friends_count': 1515,
  'listed_count': 874,
  'favourites_count': 15790,
  'statuses_count': 2599,
  'created_at': 1220639321},
 1225145123920588805: {'id': 1225145123920588805,
  'name': 'efe09',
  'screen_name': 'efe0927183508',
  'location': None,
  'url': None,
  'descri

In [18]:
import pandas as pd 
userstable= pd.DataFrame(users.values())
userstable

,id,name,screen_name,location,url,description,verified,followers_count,friends_count,listed_count,favourites_count,statuses_count,created_at
0,1242817830946508801,juwelz v,juwelz_v,"Lower East Side, Manhattan",https://linktr.ee/juwelzv,Event Lyfe LLC .. Brand Ambassador: #visionary...,False,43,118,0,722,906,1585160248
1,16144221,NUFF,nuffsaidny,None,None,instagram: @nuffsaidny 🇹🇹,False,17112,1515,874,15790,2599,1220639321
2,1225145123920588805,efe09,efe0927183508,None,None,Allah'ın en değerli eseri insandır.\nCanı yana...,False,653,983,0,1255,4177,1580950358
3,1087735689091928064,Karanfil Lale,lale_karanfil,None,None,None,False,897,1120,1,2776,7986,1548189372
4,101007632,Ravin Gupta,IamRaavin,india,None,Tweet is personal opinion and Retweet is not e...,False,499,537,2,4342,4038,1262381064
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18456,1199540529748312069,अक्षय कुमार ♻,AkshaySpeaks1,India,None,"Social Activist, Media, Bollywood, Parody, जय ...",False,4648,2437,0,2441,2940,1574845738
18457,491861073,KATHRYN NICHOLSON 🇨🇦,bucksbratbonnie,None,None,None,False,42,181,0,12952,10823,1329206827
18458,947922936631103489,CEO of Horny,pimpjuice_k,None,None,"Nobel peace prize winner, Mulit-Platinum produ...",False,1960,186,2,849,1454,1514855414
18459,884226045846704129,sophia ♡,ssophhiiiaaaa,angelo state,None,im shy and also kinda funny sometimes,False,150,96,2,36747,14106,1499665292


In [85]:
### UPLOADING DATA TO DB

In [20]:
import mysql.connector as msql
from mysql.connector import Error
try:
    conn = msql.connect(host='localhost', user='root',  
                        password='Password')#give ur username, password
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE TwitterDB")
        print("Database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

Error while connecting to MySQL 2003 (HY000): Can't connect to MySQL server on 'localhost:3306' (10061)


In [22]:
import mysql.connector as msql
try:
    conn = msql.connect(host='localhost', database='TwitterDB', user='root', password='Password')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS users_data;')
        print('Creating table....')
# in the below line please pass the create table statement which you want #to create
        cursor.execute("CREATE TABLE users_data(id BIGINT,name varchar(255),screen_name varchar(255),location varchar(255),url varchar(255),description varchar(255),verified TINYINT,followers_count int,friends_count int,listed_count int,favourites_count int,statuses_count int,created_at int)")
        print("Table is created....")
        #loop through the data frame
        for i,row in userstable.iterrows():
            #here %S means string values 
            sql = "INSERT INTO TwitterDB.users_data VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            # the connection is not auto committed by default, so we must commit to save our changes
            conn.commit()
except Error as e:
     print("Error while connecting to MySQL", e)

Error while connecting to MySQL 2003 (HY000): Can't connect to MySQL server on 'localhost:3306' (10061)


In [13]:
i =0 


{'created_at': 1586730445, 'id': 1249403767180668930, 'text': 'RT @nuffsaidny: wishing death on people is weirdo behavior.', 'hashtags': [], 'user': {'id': 1242817830946508801, 'name': 'juwelz v', 'screen_name': 'juwelz_v', 'verified': False}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted_status': {'created_at': 1586724481, 'id': 1249378751349231616, 'text': 'wishing death on people is weirdo behavior.', 'hashtags': [], 'user': {'id': 16144221, 'name': 'NUFF', 'screen_name': 'nuffsaidny', 'verified': False}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'quote_count': 1, 'reply_count': 1, 'retweet_count': 24, 'favorite_count': 50, 'is_quote_status': True, 'quoted_status': {'created_at': 1586709390, 'id': 1249315454797168641, 'text': 'I pray my haters die of corona virus 😷', 'hashtags': [], 'user': {'i

In [14]:

from pymongo import MongoClient
  
try:
    conn = MongoClient()
    print("Connected successfully!!!")
except:  
    print("Could not connect to MongoDB")
  
# database
db = conn.TwitterDB
collection = db.Tweet
  

  
# Insert Data
for key in retweets.keys():
    collection.insert_one(retweets[key])

# collection.insert_many(tweets)

  

C:\Users\mafaz\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Connected successfully!!!


In [84]:
from pymongo import MongoClient
  
try:
    conn = MongoClient()
    print("Connected successfully!!!")
except:  
    print("Could not connect to MongoDB")
  
# database
db = conn.TwitterDB
collection = db.Retweet
  

  
# Insert Data

collection.insert_many(retweets)

Connected successfully!!!
